In [1]:
import sys
import gzip
#import pyfrost
import numpy
from collections import defaultdict
import matplotlib.pyplot as plot
#import CCGG_extension as CCGG
import json
import AGG
import pandas as pd
from Bio import pairwise2
from Bio.Seq import Seq
import Levenshtein

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/Bio/pairwise2.py:283: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  BiopythonDeprecationWarning,


In [2]:
filename = './MT_anchor_graphical_genome_k31_anno_delunspported.gfa'
filename = './MT_anchor_graphical_genome_k31_anno_series_parallele.gfa'
spgraph = AGG.GraphicalGenome(filename)

In [3]:
edgelist = spgraph.edges.keys()
for edge in edgelist:
    print(spgraph.edges[edge].get('variants', ""))
    break

52=


In [4]:
#help(AGG.SubGraph)
def find_sample_list(graph):
    samplelist = []
    edgelist = graph.edges.keys()
    for edge in edgelist:
        samplelist += graph.edges[edge]['strain']
    samplelist = list(set(samplelist))
    return samplelist


In [5]:
#samplelist

In [8]:
#'A000069' in subgraph.outgoing

In [9]:
with open('/Users/suhang/Analysis/Mitochondrial/MitoHifi/Mitochondrial/NC_012920.1.fasta') as fp:
    data = fp.readlines()
data.pop(0)
contig = "".join([l[:-1] for l in data])


In [10]:
sample = '012920'
subgraph = AGG.SubGraph(spgraph, [sample,  '012920'])
path = AGG.find_most_supported_path(subgraph, [sample])
p = path[0]
contig.find(p)

23

In [11]:
# sorted(subgraph.anchor.keys())
outputfile = "test.fasta"
def write_fasta(outputfile, sequence, sample):
    header = ">" + sample + ", mitochondrial most supported path" + '\n'
    with open(outputfile, 'w') as fp:
        fp.write(header)
        l = len(sequence)//60
        for i in range(l+1):
            seq = sequence[i*60:(i+1)*60]
            fp.write(seq + '\n')
        fp.write(sequence[(i+1)*60:] + '\n')

In [12]:
samplelist = find_sample_list(spgraph)
for sample in samplelist:
    subgraph = AGG.SubGraph(spgraph, [sample,  '012920'])
    try:
        path = AGG.find_most_supported_path(subgraph, [sample])
        print(sample, len(path[0]))
        sequence = path[0]
        outputfile = "../optimal_path_from_graph/%s.fasta" % sample
        write_fasta(outputfile, sequence, sample)
    except:
        print(sample, "Fail")

NA18906 16651
HG03453 16653
HG00735 16651
HG02109 16653
012920 16407
HG02572 16654
HG02257 16653
HG02717 16651
HG03540 16650
HG00733 16651
HG01123 16651
HG02886 16651
HG01071 16651
HG002 16652
HG01243 16650
HG02559 16652
NA20129 16651
HG01928 16652
HG03098 16653
HG02622 16651
HG02080 16652
HG01891 16650
HG01175 16653
HG01106 16654
NA21309 16651
HG02148 16651
NA19240 16650
HG00673 16651
HG02818 16651
HG02723 16651
HG03579 16650
HG02630 16653
HG01952 16651
HG00621 16651
HG01358 16651
HG02055 16653
HG03516 16651
HG03492 16652
HG01109 16653
HG03486 16650
HG02145 16653
HG01258 16652
HG00438 16653
HG01978 16652
HG00741 16652
HG01361 16651
HG02486 16651


In [13]:
sample = ['HG002']
subgraph = AGG.SubGraph(spgraph, sample)
edgelist = subgraph.edges.keys()
for edge in edgelist:
    assert len(set(sample) & set(subgraph.edges[edge].get('strain', "")))>0, print(subgraph.edges[edge].get('strain', ""))

In [14]:
for edge in spgraph.outgoing['A000151']:
    if sample in (spgraph.edges[edge]['strain']):
        print(edge)

In [15]:
#AGG.write_gfa(subgraph.anchor, subgraph.edges, "./HG002_subgraph_k31.gfa")

def find_most_supported_path(graph, sample, ref = '012920'):
    src = sorted(graph.anchor.keys())[0]
    dst = sorted(graph.anchor.keys())[-1]
    Path = []

    visited = set()
    
    def find_most_supported_edge(graph, sample, src, ref):
        
        edgelist = graph.outgoing.get(src, [])
        sample_edge = []
        for edge in edgelist:
            if sample in graph.edges[edge]['strain']:
                sample_edge += [edge]
        if len(sample_edge)<1: # no haplotype for this sample at this place the go to the reference one
            for ref_edge in edgelist:
                if ref in set(graph.edges[ref_edge]['strain']):
                    break
            edge = ref_edge

        elif len(sample_edge) == 1:
            edge = sample_edge[0]
        else:
            # find the most supported edge
            read_count = [len(graph.edges[e]['reads']) for e in sample_edge]

            index = numpy.argmax(read_count)

            edge = sample_edge[index]
            
        return edge
            
    while src != dst:
        edge = find_most_supported_edge(graph, sample, src, ref)
        Path += [src, edge]

        visited.add(src)

        src = graph.outgoing[edge][0]

        assert src.startswith("A") or src == "SINK"

        if src in visited:
            break
    
    if src == dst:
        try:
            edge = find_most_supported_edge(graph, sample, src, ref)
            Path += [src, edge]
        except:
            Path = Path
        
    reconstruct = ''
    for item in Path:
        if item == "SOURCE":
            reconstruct += ""
        elif item.startswith('A'):
            reconstruct += graph.anchor[item]['seq']
        elif item.startswith("E"):
            reconstruct += graph.edges[item]['seq']
        else:
            print(item)

    return reconstruct, Path

In [16]:
# Extract most supported path
path = AGG.find_most_supported_path(subgraph, sample)

In [17]:
# call major haplotype variants


In [18]:
def get_variant_position(cigar):
    ref_pos = []
    alt_pos = []
    var_type = []
    alt_i = 0
    ref_i = 0
    for i, s in enumerate(cigar):
        if s == 'I':
            if ref_i > 0:
                ref_pos.append(ref_i-1)
            else:
                ref_pos.append(ref_i)    
            alt_pos.append(alt_i)
            var_type.append("I")
            
            alt_i += 1
            
        if s == 'D':
            ref_pos.append(ref_i)
            if alt_i > 0:
                alt_pos.append(alt_i-1)
            else:
                alt_pos.append(alt_i)
            var_type.append("D")
            ref_i += 1
            
        if s == 'X':
            ref_pos.append(ref_i)
            alt_pos.append(alt_i)
            alt_i += 1
            ref_i += 1
            var_type.append("X")
            
        if s == '=':
            alt_i += 1
            ref_i += 1

    return ref_pos, alt_pos, var_type

def findBedge(Graph, src, dst, refstrain, k):
    paths = AGG.Find_all_Path_between_anchors(Graph, src, dst, {refstrain})
    subpaths = paths.subpath

    if len(subpaths) < 1:
        return ""
    for p, strain in subpaths:
        seq = AGG.reconstruct_path_seq(spgraph, path = p)
    return p[1:-1], seq[k:-k]

In [19]:
# for edge in spgraph.outgoing[src]:
#     if ref in spgraph.edges[edge]['strain']:
#         print(spgraph.edges[edge]['seq'])

In [20]:
# print(src, dst)
# ref_seq = findBedge(spgraph, src, dst, "NC_012920", k)
# ref_seq

In [21]:
# Find variants along the path


def get_snps(subgraph, mpath, k, ref):
    SNPs = {} # Var[refpos]['edgename'] = ['A']
    for edge in mpath:
        if edge.startswith('E'):
            cigar = subgraph.edges[edge].get('variants', "")
            if cigar == "":
                continue
            src = subgraph.incoming[edge][0]
            if src != "SOURCE":
                refstart = int(subgraph.anchor[src]["pos"]) + k
            else:
                raise "SOURCE or SINK node"

            # find reference seq
            dst = subgraph.outgoing[edge][0]
            ref_edge_list, ref_seq = findBedge(spgraph, src, dst, "NC_012920", k)
            ref_edge = ref_edge_list[0]
            expanded_cigar = AGG.processCigar(cigar)
            refpos, altpos, var_type = get_variant_position(AGG.processCigar(cigar))
            alt_seq = subgraph.edges[edge]['seq']
            # only record SNPs
            for i, vart in enumerate(var_type):
                if vart == "X":
                    rp = refpos[i]
                    ap = altpos[i]
                    pos = refstart + rp
                    SNPs[pos] = SNPs.get(pos, {})
                    SNPs[pos][edge] = SNPs[pos].get(edge, {})                
                    SNPs[pos][edge]['base'] = SNPs[pos][edge].get('base', "") + alt_seq[ap]
                    SNPs[pos][edge]['altoffset'] = SNPs[pos][edge].get('altoffset', []) + [ap]
                    SNPs[pos][ref_edge] = SNPs[pos].get(ref_edge, {})
                    SNPs[pos][ref_edge]['base'] = ref_seq[refpos[i]]
                    SNPs[pos]['refbase'] = SNPs[pos][ref_edge]['base']
    return SNPs
#     node = graph.outgoing[edge][0]

In [22]:
# def find_allVar(graph, refstrain, founder, k):
#     #print cigar, refpos, altpos
#     return Var

# # Single Sample VCF

def get_VCF_file(Var, samplelist):
    Mat = {}

    for pos, D in Var.items():
        Mat[pos] = {}
        for edge, B in D.items():
            if edge == "refbase":
                continue
            for strain in samplelist:
                Mat[pos][strain] = Mat[pos].get(strain, set()) | set([B['base']])
    # write VCF
    VCF = {}
    for pos, D in Mat.items():
        refbase = Var[pos]['refbase']
        
        if list(D.values())[0] == set([refbase]):
            continue    
        VCF[pos]= VCF.get(pos, {})
        for strain, base in D.items():
            VCF[pos]["CHROM"] = "chrM"
            VCF[pos]["POS"] = pos
            VCF[pos]['ID'] = "."
            VCF[pos]['REF'] = refbase
            #print(base - set([refbase]))
            VCF[pos]['ALT'] = ",".join(base - set([refbase]))
            VCF[pos]['QUAL'] = "."
            VCF[pos]['FILTER'] = "."
            VCF[pos]['INFO'] = "."# Indels[pos][edge]['info'] = "INS"
            VCF[pos]['FORMAT'] = "GT"
            #print(base)
            # allelelist = []
            # if refbase in base:
            #     allelelist += [0]

            # for i,s in enumerate(base - set([refbase])):
            #     allelelist += [i+1]
            # VCF[pos][strain] = "/".join([str(item) for item in allelelist])
            VCF[pos][strain] = "/".join([str(item) for item in base])
    return VCF

In [23]:
sample

['HG002']

In [24]:
mpath = path[1]
k = 31
ref = "012920"
SNPs = get_snps(subgraph, mpath, k, ref)
VCF_SNPS = get_VCF_file(SNPs, [sample])
df = pd.DataFrame(VCF_SNPS).T
df

TypeError: unhashable type: 'list'

In [25]:
# main function for calling major SNPs
k = 31
refstrain = "012920"
# samplelist = ['HG00438', "HG00621"]
#samplelist = ["HG002",'HG00621', "HG00438"]
for sample in samplelist:
    subgraph = AGG.SubGraph(spgraph, [sample,  '012920']) # always keep the reference
    path = AGG.find_most_supported_path(subgraph, sample)
    mpath = path[1]
    SNPs = get_snps(subgraph, mpath, k, ref)
    VCF_SNPS = get_VCF_file(SNPs, [sample])
    df = pd.DataFrame(VCF_SNPS).T
    print(sample, df.shape)
    #df.to_csv("../optimal_path_from_graph/%s.snps.vcf" % sample, index = False, sep = '\t')

NA18906 (80, 10)
HG03453 (49, 10)
HG00735 (39, 10)
HG02109 (82, 10)
012920 (0, 0)
HG02572 (77, 10)
HG02257 (39, 10)
HG02717 (55, 10)
HG03540 (55, 10)
HG00733 (28, 10)
HG01123 (27, 10)
HG02886 (48, 10)
HG01071 (27, 10)
HG002 (12, 10)
HG01243 (77, 10)
HG02559 (21, 10)
NA20129 (29, 10)
HG01928 (24, 10)
HG03098 (50, 10)
HG02622 (30, 10)
HG02080 (34, 10)
HG01891 (54, 10)
HG01175 (29, 10)
HG01106 (34, 10)
NA21309 (52, 10)
HG02148 (30, 10)
NA19240 (35, 10)
HG00673 (29, 10)
HG02818 (38, 10)
HG02723 (40, 10)
HG03579 (55, 10)
HG02630 (53, 10)
HG01952 (32, 10)
HG00621 (20, 10)
HG01358 (32, 10)
HG02055 (34, 10)
HG03516 (49, 10)
HG03492 (29, 10)
HG01109 (82, 10)
HG03486 (52, 10)
HG02145 (54, 10)
HG01258 (26, 10)
HG00438 (31, 10)
HG01978 (5, 10)
HG00741 (79, 10)
HG01361 (27, 10)
HG02486 (10, 10)


In [26]:
def get_indels(subgraph, mpath, k, ref):
    Indels = {} # Var[refpos]['edgename'] = ['A']
    for edge in mpath:
        if edge.startswith('E'):
            cigar = subgraph.edges[edge].get('variants', "")
            if cigar == "":
                continue
            src = subgraph.incoming[edge][0]
            if src != "SOURCE":
                refstart = int(subgraph.anchor[src]["pos"]) + k
            else:
                raise "SOURCE or SINK node"

            # find reference seq
            dst = subgraph.outgoing[edge][0]
            ref_edge_list, ref_seq = findBedge(spgraph, src, dst, "NC_012920", k)
            ref_edge = ref_edge_list[0]
            expanded_cigar = AGG.processCigar(cigar)
            refpos, altpos, var_type = get_variant_position(AGG.processCigar(cigar))
            alt_seq = subgraph.edges[edge]['seq']
            # only record SNPs
            for i, vart in enumerate(var_type):
                if vart == "I":
                    rp = refpos[i]
                    ap = altpos[i]
                    pos = refstart + rp
                    Indels[pos] = Indels.get(pos, {})
                    Indels[pos][edge] = Indels[pos].get(edge, {})                
                    Indels[pos][edge]['base'] = Indels[pos][edge].get('base', "") + alt_seq[ap:ap+2]
                    Indels[pos][edge]['info'] = "INS"
                    Indels[pos][edge]['altoffset'] = Indels[pos][edge].get('altoffset', []) + [ap]
                    Indels[pos][ref_edge] = Indels[pos].get(ref_edge, {})
                    Indels[pos][ref_edge]['base'] = Indels[pos][ref_edge].get("base", "") + ref_seq[rp-1]
                    Indels[pos]['refbase'] = Indels[pos][ref_edge]['base']

                if vart == "D":
                    rp = refpos[i]
                    ap = altpos[i]
                    pos = refstart + rp
                    Indels[pos] = Indels.get(pos, {})
                    Indels[pos][edge] = Indels[pos].get(edge, {})                
                    Indels[pos][edge]['base'] = Indels[pos][edge].get('base', "") + alt_seq[ap]
                    Indels[pos][edge]['info'] = "DEL"
                    Indels[pos][edge]['altoffset'] = Indels[pos][edge].get('altoffset', []) + [ap-1]
                    Indels[pos][ref_edge] = Indels[pos].get(ref_edge, {})
                    Indels[pos][ref_edge]['base'] = Indels[pos][ref_edge].get("base", "") + ref_seq[rp-1:rp+1]
                    Indels[pos]['refbase'] = Indels[pos][ref_edge]['base']
    return Indels

In [27]:
k = 31
refstrain = "012920"
# samplelist = ['HG00438', "HG00621"]
#samplelist = ["HG002",'HG00621', "HG00438"]
for sample in ["HG002"]:
    subgraph = AGG.SubGraph(spgraph, [sample,  '012920']) # always keep the reference
    path = AGG.find_most_supported_path(subgraph, sample)
    mpath = path[1]
    Indels = get_indels(subgraph, mpath, k, ref)
    VCF_SNPS = get_VCF_file(Indels, [sample])
    df = pd.DataFrame(VCF_SNPS).T
    print(sample, df.shape)
    #df.to_csv("../optimal_path_from_graph/%s.indels.vcf" % sample, index = False, sep = '\t')

HG002 (3, 10)


In [28]:
df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,HG002
309,chrM,309,.,C,CT,.,.,.,GT,C/CT
310,chrM,310,.,C,CC,.,.,.,GT,C/CC
3107,chrM,3107,.,CN,C,.,.,.,GT,CN/C


# call major and heteroplasmy from the whole subgraph

In [49]:
15 * 0.05

0.75

In [50]:
# def prune_under_supported_path(spgraph, samplelist, threshold = 0.05):
#     for sample in samplelist:
#         sub_spgraph = AGG.SubGraph(spgraph, [sample])
#         nodelist = sorted(sub_spgraph.outgoing.keys())    
#         readsets = AGG.find_all_reads(sub_spgraph)
#         readnum = sum([1 for r in list(readsets) if r.endswith(sample)])
#         print(readnum)
#         for i in range(len(nodelist)-1):
#             sanchor = nodelist[i]
#             eanchor = nodelist[i+1]
#             paths = sub_spgraph.outgoing[sanchor]
#             supported = [p for p in paths if p.startswith('E') if len(set(sub_spgraph.edges[p]['reads'])) > readnum * threshold]
#             unsupported = [p for p in paths if p not in supported]
#             # delete unsupported
#             for e in unsupported:
#                 if e.startswith("E"):
#                     del sub_spgraph.edges[e]

#     return sub_spgraph

In [87]:
threshold = 0.05
k = 31
refstrain = "012920"
# samplelist = ['HG00438', "HG00621"]
#samplelist = ["HG002",'HG00621', "HG00438"]
for sample in ['HG002']:
    subgraph = AGG.SubGraph(spgraph, [sample,  '012920']) # always keep the reference
    readnum = [read for read in AGG.find_all_reads(subgraph) if read.endswith(sample)]
    
    #sub_graph = prune_under_supported_path(subgraph, [sample,  '012920'], threshold = 0.005)
    #path = AGG.find_most_supported_path(subgraph, sample)
    paths = list(subgraph.edges.keys())
    mpath = [edge for edge in paths if len([r for r in subgraph.edges[edge]['reads'] if r.endswith(sample)]) > len(readnum) * threshold]
    SNPs = get_snps(subgraph, mpath, k, ref)
    Indels = get_indels(subgraph, mpath, k, ref)
    VCF_SNPS = get_VCF_file(SNPs, [sample])
    VCF_Indels = get_VCF_file(Indels, [sample])
    df = pd.DataFrame(VCF_SNPS).T
    di = pd.DataFrame(VCF_Indels).T
    print(sample, df.shape, di.shape)
    #df.to_csv("../optimal_path_from_graph/%s.indels.vcf" % sample, index = False, sep = '\t')

HG002 (12, 10) (131, 10)


In [88]:
for edge in mpath:
    print(subgraph.edges[edge]['variants'])

52=
125=1X46=2I1=1I56=
125=1X46=1I1=1I56=
125=1X46=2I1=2I56=
125=1X47=1I56=
125=1X46=3I1=1I56=
58=1X1=
124=
106=1X82=
106=1X51=1D30=
64=1D41=1X82=
94=
39=1I55=
94=1I
34=1D59=
10=1D83=
98=
9=1D88=
66=
51=1I15=
10=1I56=
108=
24=1D83=
11=1D96=
84=1X48=
70=
26=1I44=
135=
67=1I68=
94=1D40=
107=1D27=
84=
1=1D82=
59=
71=
59=1D11=
38=
19=1I19=
100=
58=1D41=
62=
40=1D21=
76=
31=1I45=
79=
53=1I26=
72=
95=
65=1I30=
89=1D5=
77=
74=1D2=
89=
48=1D44=
19=1D28=1D44=
22=1I26=1D44=
79=
51=1D27=
67=
204=
181=1I23=
188=1I16=
175=1I29=
118=
21=1I97=
53=
41=1D11=
16=1I37=
116=
64=
15=1I49=
166=
40=1D125=
38=1X57=
38=1X29=1I28=
38=1X29=1D27=
108=
38=1D69=
14=1D93=
103=
71=1X279=
71=1X277=1I2=
71=1X155=1D123=
71=1X100=1I179=
47=1D23=1X279=
27=1I44=1X155=1I62=1D61=
86=
72=1I14=
137=
90=1D46=
39=1D97=
106=
76=1I30=
46=1D59=
231=
74=1D156=
90=
70=1D19=
78=
31=1D46=
28=1I50=
72=
127=
76=1D50=
93=1I34=
92=1D34=
152=
149=
134=1D14=
13=1I115=1I21=
64=1D84=
46=
15=1D30=
60=
15=1I45=
14=1X117=
260=
244=1D15=
196=
110=

KeyError: 'variants'

In [89]:
di

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,HG002
309,chrM,309,.,CCCCCCCC,"CCCT,CT,CCCCCT",.,.,.,GT,CCCCCT/CCCT/CT/CCCCCCCC
310,chrM,310,.,CCCCCC,"CCCC,CC",.,.,.,GT,CCCC/CC/CCCCCC
802,chrM,802,.,AC,A,.,.,.,GT,AC/A
708,chrM,708,.,CC,C,.,.,.,GT,CC/C
902,chrM,902,.,G,GT,.,.,.,GT,G/GT
...,...,...,...,...,...,...,...,...,...,...
15682,chrM,15682,.,AA,A,.,.,.,GT,AA/A
15540,chrM,15540,.,CC,C,.,.,.,GT,CC/C
15411,chrM,15411,.,C,CT,.,.,.,GT,C/CT
15884,chrM,15884,.,GG,G,.,.,.,GT,G/GG


In [93]:
# traverse the graph
# from chat-gpt
# def dfs(graph, start, end, reads):
#     stack = [(start, [start], reads)]
#     while stack:
#         (node, path, reads) = stack.pop()
#         for next_node, next_reads in graph[node].items():
#             intersect_reads = reads.intersection(next_reads)
#             if not intersect_reads:
#                 continue
#             if next_node == end:
#                 yield path + [next_node]
#             else:
#                 stack.append((next_node, path + [next_node], intersect_reads))

# # Example usage:
# graph = {
#     'A': {'B': {'read1', 'read2'}, 'C': {'read2', 'read3'}},
#     'B': {'A': {'read1'}, 'D': {'read4'}, 'E': {'read5'}},
#     'C': {'A': {'read2'}, 'F': {'read3'}},
#     'D': {'B': {'read4'}},
#     'E': {'B': {'read5'}, 'F': {'read6'}},
#     'F': {'C': {'read3'}, 'E': {'read6'}},
# }
# start = 'A'
# end = 'F'
# reads = {'read1', 'read2', 'read3'}

# paths = list(dfs(graph, start, end, reads))
# for path in paths:
#     print(path)

In [ ]:
# def dfs(graph, node, end, path, reads):
#     path = path + [node]
#     if node == end:
#         return [path]
#     paths = []
#     for next_node, next_reads in graph[node].items():
#         intersect_reads = reads.intersection(next_reads)
#         if not intersect_reads or next_node in path:
#             continue
#         newpaths = dfs(graph, next_node, end, path, intersect_reads)
#         for newpath in newpaths:
#             paths.append(newpath)
#     return paths

# # Example usage:
# graph = {
#     'A': {'B': {'read1', 'read2'}, 'C': {'read2', 'read3'}},
#     'B': {'A': {'read1'}, 'D': {'read4'}, 'E': {'read5'}},
#     'C': {'A': {'read2'}, 'F': {'read3'}},
#     'D': {'B': {'read4'}},
#     'E': {'B': {'read5'}, 'F': {'read6'}},
#     'F': {'C': {'read3'}, 'E': {'read6'}},
# }
# start = 'A'
# end = 'F'
# reads = {'read1', 'read2', 'read3'}

# paths = dfs(graph, start, end, [], reads)
# for path in paths:
#     print(path)